# Regresión de Pagos en Pruebas A/B

Este estudio de caso está tomado de este [Repositorio de GitHub](https://github.com/matheusfacure/python-causality-handbook/tree/master).

La tarea es determinar el impacto de enviar un correo electrónico que invita a las personas a negociar su deuda. El outcome de interés es la cantidad de pagos realizados por los clientes morosos.

Después de seguir los pasos 1 - 4 del notebook anterior sobre pruebas A/B, el equipo de ciencia de datos selecciona aleatoriamente a 5,000 clientes de tu grupo de clientes morosos para realizar una prueba aleatoria.

Cada cliente recibe el correo electrónico al azar o es colocado en el grupo de control. El objetivo es descubrir cuánto dinero adicional genera el correo electrónico.

**Diccionario de datos:**

- payments: cantidad de pagos realizados por el cliente
- email: si el cliente recibió el correo electrónico (1) o no (0)
- opened: si el cliente abrió el correo electrónico (1) o no (0)
- agreement: si el cliente contactó al departamento de cobranzas para negociar la deuda (1) o no (0), después de recibir el correo electrónico
- credit_limit: la línea de crédito del cliente antes de retrasarse en los pagos
- risk_score: puntaje de riesgo estimado del cliente antes de recibir el correo electrónico

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import scipy.stats as stats

## Check the data

In [ ]:
# Read data
path_data = 'https://github.com/pabloestradac/experimentation-notebooks/raw/main/data/'
df = pd.read_csv(path_data + 'collections_email.csv')
df.head()

,payments,email,opened,agreement,credit_limit,risk_score
0,740,1,1.0,0.0,2348.495260,0.666752
1,580,1,1.0,1.0,334.111969,0.207395
2,600,1,1.0,1.0,1360.660722,0.550479
3,770,0,0.0,0.0,1531.828576,0.560488
4,660,0,0.0,0.0,979.855647,0.455140


In [ ]:
# Descriptive Statistics
df.describe()

,payments,email,opened,agreement,credit_limit,risk_score
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,669.672000,0.490800,0.273400,0.160800,1194.845188,0.480812
std,103.970065,0.499965,0.445749,0.367383,480.978996,0.100376
min,330.000000,0.000000,0.000000,0.000000,193.695573,0.131784
25%,600.000000,0.000000,0.000000,0.000000,843.049867,0.414027
50%,670.000000,0.000000,0.000000,0.000000,1127.640297,0.486389
75%,730.000000,1.000000,1.000000,0.000000,1469.096523,0.552727
max,1140.000000,1.000000,1.000000,1.000000,3882.178408,0.773459


## Regresión para Pruebas A/B

¿Cuál es el efecto de enviar un correo electrónico a los clientes morosos sobre la cantidad de pagos realizados?

$$
\text{Payments} = \beta_0 + \beta_1 \times \text{Email} + \epsilon
$$

In [ ]:
model_base = 'payments ~ email'
base = smf.ols(model_base, data=df)
results_base = base.fit()
print(results_base.summary())

                            OLS Regression Results                            
Dep. Variable:               payments   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                   0.04447
Date:                Wed, 30 Apr 2025   Prob (F-statistic):              0.833
Time:                        00:40:30   Log-Likelihood:                -30315.
No. Observations:                5000   AIC:                         6.063e+04
Df Residuals:                    4998   BIC:                         6.065e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    669.9764      2.061    325.116      0.0

In [ ]:
base_est = results_base.params['email']
base_se = results_base.bse['email']
base_ci = results_base.conf_int().loc['email']
print(f"The effect of the treatment is: {base_est:.2f}")
print(f"Standard error: {base_se:.4f}")
print(f"Confidence interval: [{base_ci[0]:.2f}, {base_ci[1]:.2f}]")

The effect of the treatment is: -0.62
Standard error: 2.9415
Confidence interval: [-6.39, 5.15]


¿Qué puedes concluir? ¿Enviar un correo electrónico aumenta la cantidad de pagos realizados por los clientes morosos? ¿Es esta diferencia estadísticamente significativa?

- Enviar un correo electrónico disminuye la cantidad de pagos realizados en \$0.6 en promedio. Sin embargo, esta diferencia no es estadísticamente significativa.

## Comparación de los grupos

¿Son similares los grupos en términos de límite de crédito y puntaje de riesgo?

In [ ]:
df['high_risk'] = df['risk_score'] > 0.5
df['high_credit'] = df['credit_limit'] > 1200
payment_grouped = df.groupby(['high_risk', 'high_credit', 'email']).agg({'payments': 'mean'}).unstack()
payment_grouped['diff'] = payment_grouped['payments'][1] - payment_grouped['payments'][0]
payment_grouped.round(0).astype(int)

In [ ]:
# Compare the average payments between treatment and control groups for low-risk customers
t_results =
print(f"t-statistic: {t_results.statistic:.4f} with p-value: {t_results.pvalue:.4f}")

In [ ]:
# Compare the average payments between treatment and control groups for low-credit customers
t_results =
print(f"t-statistic: {t_results.statistic:.4f} with p-value: {t_results.pvalue:.4f}")

In [ ]:
# The same as before (low-risk) but better
model_base =
base =
results_base =
print(results_base.summary().tables[1])

In [ ]:
# The same as before (low-credit) but better
model_base =
base =
results_base =
print(results_base.summary().tables[1])

Si una variable es un buen predictor del resultado, explicará gran parte de su varianza.

Si el puntaje de riesgo y el límite de crédito son fuertes predictores del comportamiento de pago, controlarlos puede ayudar a aislar el efecto del correo electrónico en los pagos.

Por lo tanto, si comparamos a los clientes con riesgos y límites de crédito similares, la variación en las cantidades de pago debería reducirse.

En otras palabras, si el puntaje de riesgo y el límite de crédito predicen con precisión los niveles de pago, los clientes con puntajes de riesgo y límites de crédito similares tendrán comportamientos de pago más consistentes, lo que resultará en menos variabilidad.

In [ ]:
model_add = 'payments ~ email + risk_score + credit_limit'
add = smf.ols(model_add, data=df)
results_add = add.fit()
print(results_add.summary().tables[1])

                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      490.8653      9.715     50.527      0.000     471.820     509.911
email            4.4304      2.130      2.080      0.038       0.255       8.606
risk_score      -8.0516     38.424     -0.210      0.834     -83.379      67.276
credit_limit     0.1511      0.008     18.833      0.000       0.135       0.167


In [ ]:
add_est =
add_se =
add_ci =
print(f"The effect of the treatment is: {add_est:.2f}")
print(f"Standard error: {add_se:.4f}")
print(f"Confidence interval: [{add_ci[0]:.2f}, {add_ci[1]:.2f}]")

In [ ]:
df['risk_dmn'] =
df['credit_dmn'] =
model_int =
inter =
results_int =
print(results_int.summary().tables[1])

In [ ]:
int_est =
int_se =
int_ci =
print(f"The effect of the treatment is: {int_est:.2f}")
print(f"Standard error: {int_se:.4f}")
print(f"Confidence interval: [{int_ci[0]:.2f}, {int_ci[1]:.2f}]")

¿Qué sucede cuando añadimos las variables `open` y `agreement` al modelo?

In [ ]:
model_add =
add =
results_add =
print(results_add.summary().tables[1])